In [3]:
# conda install pytorch torchvision -c pytorch

In [ ]:
import numpy as np
import pandas as pd
import torch
import re

from transformers import AutoModel, AutoTokenizer, RobertaForSequenceClassification

In [ ]:
df = pd.read_csv(r"C:\2024\DA\Project\[Project 4] Sentiment Analysis\mongodb_comments.csv")

df

In [ ]:
df.info()

In [11]:
def get_id_text(text):
    result = re.findall(r"\'\$oid\'\:\s*\'([^\']+)\'", text)[0]
    
    return result

In [ ]:
# Xử lý cột _id
df['_id'] = df['_id'].apply(get_id_text)

df['_id']

In [ ]:
# Chọn ra các cột cần lấy để phân tích
df = df[['_id', 'user_id', 'comment_on', 'object_id', 'content', 'device', 'timestamp']]

df

## Pre-Process Data

In [ ]:
test = """
Phim quá hay luôn...mong từng tập!
"""

def standardize_data(row):
    # Viết thường toàn bộ
    row = row.lower()
    
    # Xoá tất cả các dấu ?, !, . ở cuối câu
    row = re.sub(r'[?!\.]+\s*$', '', row)
    
    # Bỏ đi các dấu khng được cho phép
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ") \
        .replace('\\', ' ').replace(')', ' ').replace('(', ' ').replace('@', ' ')
    
    # Thay thế 1 số từ viết tắt trong tiếng việt
    row = re.sub('ko', 'không ', row)
    row = re.sub('k ', 'không ', row)
    row = re.sub('ad', 'admin ', row)
    row = re.sub('[dđ]c', 'được ', row)
    row = re.sub('thúi', 'thối ', row)
    row = re.sub('chuong trinh', 'chương trình ', row)
    row = re.sub('qua', 'quá ', row)
    row = re.sub("vn","việt nam", row)
    row = re.sub("roi","rồi", row)
    
    # Loại bỏ các dấu cách thừa
    row = re.sub(r'\s+', ' ', row).strip()
    
    return row

# standardize câu test
print(standardize_data(test))

In [ ]:
# Chuyển cột content về dạng string
df['content'] = df['content'].astype(str)

# Tiến hành apply hàm standardize_data => sau có thời gian bổ sung sửa thêm các từ viết tắt
df['content_std'] = df['content'].apply(standardize_data)

df

In [ ]:
# load pre trained model
model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")

# load Tokenizer từ model (dùng để tách từ trong câu)
tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)

In [ ]:
# test câu
sentence = 'cũng được'

# tách câu thành list từng từ
input_ids = torch.tensor([tokenizer.encode(sentence)])

with torch.no_grad():
    out = model(input_ids)
    # predict output bằng pretrained model
    out = pd.Series(out.logits.softmax(dim=-1).tolist()[0], index= ['NEG', 'POS', 'NEU'])
    print(out)

In [ ]:
# hàm để đánh giá từng comment. 0 là tiêu cực, 1 là tích cực, 2 là trung tính hoặc ko thể xác định
def sentiment(sentence):
    try:
        input_ids = torch.tensor([tokenizer.encode(sentence)])
    except:
        return 2

    if input_ids.shape[1] > tokenizer.model_max_length:
        return 2

    with torch.no_grad():
        out = model(input_ids)
        return np.argmax(out.logits.softmax(dim=-1).tolist()[0])

print(sentiment(sentence))

In [ ]:
# kiểm tra xem mô hình chạy bao thời gian
import time

start_time = time.time()

test = df[:999]

test['sentiment'] = test['content_std'].apply(sentiment)

test

In [ ]:
# chạy thật và tính ra sentiment
df['sentiment'] = df['content_std'].apply(sentiment)

df

In [55]:
df.to_csv(r'C:\2024\DA\Project\[Project 4] Sentiment Analysis\Sentiment_Result.csv', index=False, encoding='utf-8')